# HW1 Problem 2cd

This notebook reformulates the set covering problem by **limiting the number of stations to at most two** and **maximizing the number of covered road links**.

Define the set of road links
$$
R = \{ (1,2), (1,4), (4,5), (2,5), (2,3), (3,5), (3,6), (5,6) \}.
$$

Introduce a binary variable $y_{ij}$ such that
$$
y_{ij} =
\begin{cases}
1, & \text{if link } (i,j) \text{ is covered}, \\
0, & \text{otherwise},
\end{cases}
\qquad \forall (i,j) \in R.
$$

### Integer Linear Programming Formulation

**Objective:**
$$
\max \sum_{(i,j) \in R} y_{ij}
$$

**Subject to:**

- Link coverage constraints:
$$
y_{ij} \le x_i + x_j \quad \forall (i,j) \in R
$$

- Station limit constraint:
$$
\sum_{i \in N} x_i \le 2
$$

- Binary constraints:
$$
x_i \in \{0,1\} \quad \forall i \in N
$$
$$
y_{ij} \in \{0,1\} \quad \forall (i,j) \in R
$$


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po


In [2]:
N = set(range(1, 7))

R = {
    (1, 2): 1,
    (1, 4): 1,
    (4, 5): 1,
    (2, 5): 1,
    (2, 3): 1,
    (5, 3): 1,
    (5, 6): 1,
    (3, 6): 1,
}

C = {
    1: 40,
    2: 65,
    3: 43,
    4: 48,
    5: 72,
    6: 36,
}

In [3]:
m = pe.ConcreteModel()

m.nodes = pe.Set(initialize=N)
m.routes = pe.Set(initialize=R.keys(), dimen=2)

m.x = pe.Var(m.nodes, within=pe.Binary)
m.y = pe.Var(m.routes, within=pe.Binary)

m.obj = pe.Objective(
    expr=sum(m.y[i, j] for i, j in m.routes),
    sense=pe.maximize
)

def link_cover_rule(m, i, j):
    return m.y[i, j] <= m.x[i] + m.x[j]

m.cover = pe.Constraint(m.routes, rule=link_cover_rule)

m.station_limit = pe.Constraint(
    expr=sum(m.x[i] for i in m.nodes) <= 2
)

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [4]:
opt = po.SolverFactory('gurobi', solver_io='python')
results = opt.solve(m, tee=True)

m.display()

Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.2.0 25C56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 14 columns and 30 nonzeros (Max)
Model fingerprint: 0x2684f84d
Model has 8 linear objective coefficients
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]

Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 30 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)

Root relaxation: objective 6.000000e+00, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.00000    0    8   -0.00000    6.00000      -   

In [5]:
stations = [i for i in m.nodes if pe.value(m.x[i]) > 0.5]
covered = [(i, j) for i, j in m.routes if pe.value(m.y[i, j]) > 0.5]
uncovered = [(i, j) for i, j in m.routes if pe.value(m.y[i, j]) < 0.5]

print("Stations placed at nodes:", stations)
print("Covered routes:", covered)
print("Uncovered routes:", uncovered)
print("Number of covered routes:", len(covered))

Stations placed at nodes: [2, 5]
Covered routes: [(1, 2), (4, 5), (2, 5), (2, 3), (5, 3), (5, 6)]
Uncovered routes: [(1, 4), (3, 6)]
Number of covered routes: 6
